# PRÁCTICA B2-2 #

## MÓDULO DE GESTIÓN DE RIESGOS ##
### Escenarios de Estrés y Cambios de Régimen de Mercado ###

### Datos básicos: ###
- Práctica en grupos de dos personas
- Entrega el día 15 de febrero a través del aula virtual.
- Los entregables son un notebook de Python y un resumen ejecutivo en formato PDF.

### Objetivo de la práctica ###
El objetivo de esta práctica es rediseñar un motor de stress testing en Python capaz de
capturar el riesgo de cola y los cambios de régimen, identificar cuándo el mercado entra en
“crisis” y cuantificar el riesgo real cuando la diversificación desaparece. El motor de
simulación deberá utilizarse explícitamente para construir Escenarios de Estrés cuyo
objetivo sea “romper la cartera”, forzando condiciones adversas y económicamente
coherentes, y cuantificando pérdidas extremas mediante VaR del 99% y Expected Shortfall
(CVaR).

### Fase 0 - Preparacion y Estructura del Proyecto ###

### Librerias ###

In [86]:
from dataclasses import dataclass, field
from datetime import date
from pathlib import Path
from typing import Dict, List, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from hmmlearn import hmm
from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from scipy import stats as sp_stats

### Variables ###

In [ ]:
RANDOM_SEED = 42

BASE_DIR = Path("..").resolve()
DATA_DIR = BASE_DIR / "data"
DATA_BRONZE_DIR = DATA_DIR / "bronze"
DATA_SILVER_DIR = DATA_DIR / "silver"
DATA_GOLD_DIR = DATA_DIR / "gold"
FIGURES_DIR = BASE_DIR / "figures"
REPORT_DIR = BASE_DIR / "report"

START_DATE = "2006-01-01"
END_DATE = date.today().isoformat()

COMBINED_PATH = DATA_GOLD_DIR / "market_data_combined.csv"

### Clases ###

In [88]:
@dataclass
class MarketData:
    """Utility class for downloading and combining market data."""

    equities: List[str] = field(default_factory=list)
    yields: List[str] = field(default_factory=list)

    combined_data: pd.DataFrame = field(init=False)

    def __post_init__(self) -> None:
        equity_data = (
            self.fetch_equities(self.equities, start=START_DATE, end=END_DATE)
            if self.equities
            else pd.DataFrame()
        )
        yield_data = (
            self.fetch_us_yields(self.yields, start=START_DATE, end=END_DATE)
            if self.yields
            else pd.DataFrame()
        )
        self.combined_data = self.combine_and_fill(equity_data, yield_data)

    @staticmethod
    def fetch_equities(tickers: List[str], start: str, end: str) -> pd.DataFrame:
        """Fetch adjusted close prices for a list of tickers using yfinance."""
        if not tickers:
            return pd.DataFrame()

        equities = yf.download(
            tickers,
            start=start,
            end=end,
            progress=False,
            threads=True,
            auto_adjust=True,
        )["Close"]

        if isinstance(tickers, list):
            tickers_join = "_".join(tickers)
        else:
            tickers_join = str(tickers)

        DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
        equities_path = DATA_BRONZE_DIR / f"equities_adj_close_{tickers_join}.csv"
        equities.sort_index().to_csv(equities_path)

        return equities.sort_index()

    @staticmethod
    def fetch_us_yields(tickers: Union[List[str], str], start: str, end: str) -> pd.DataFrame:
        """Fetch US yields from FRED."""
        if not tickers:
            return pd.DataFrame()

        yields = pdr.DataReader(tickers, "fred", start, end)
        yields.index = pd.to_datetime(yields.index)

        if isinstance(tickers, list):
            tickers_join = "_".join(tickers)
        else:
            tickers_join = str(tickers)

        DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
        yields_path = DATA_BRONZE_DIR / f"us_yields_{tickers_join}.csv"
        yields.sort_index().to_csv(yields_path)

        return yields.sort_index()

    @staticmethod
    def combine_and_fill(equities: pd.DataFrame, yields: pd.DataFrame) -> pd.DataFrame:
        """Combine equities and yields into a single DataFrame and forward-fill missing yield data."""
        combined = pd.concat([equities, yields], axis=1).sort_index()

        # Forward-fill only yield series to avoid contaminating equity prices
        for col in ["GS10", "GS2", "BAMLH0A0HYM2"]:
            if col in combined.columns:
                combined[col] = combined[col].ffill()

        DATA_SILVER_DIR.mkdir(parents=True, exist_ok=True)
        combined_path = DATA_SILVER_DIR / "market_data_combined.csv"
        combined.to_csv(combined_path)

        return combined


@dataclass
class Portfolio:
    """Equal-weight portfolio built from equities and yield instruments."""

    assets: Dict[str, str]

    prices: pd.DataFrame = field(init=False)
    returns: pd.DataFrame = field(init=False)
    weights: pd.DataFrame = field(init=False)
    portfolio_returns: pd.Series = field(init=False)

    def __post_init__(self) -> None:
        self._load_prices()
        self._compute_returns()
        self._compute_dynamic_weights()
        self._compute_portfolio_returns()

    def _load_prices(self) -> None:
        equities = [
            ticker for ticker, asset_type in self.assets.items() if asset_type == "equity"
        ]
        yields = [
            ticker for ticker, asset_type in self.assets.items() if asset_type == "yield"
        ]

        equity_data = MarketData.fetch_equities(equities, start=START_DATE, end=END_DATE)
        yield_data = MarketData.fetch_us_yields(yields, start=START_DATE, end=END_DATE)

        self.prices = MarketData.combine_and_fill(equity_data, yield_data)

    def _compute_returns(self) -> None:
        self.returns = self.prices.pct_change()

    def _compute_dynamic_weights(self) -> None:
        asset_exists = ~self.prices.isna()
        n_assets = asset_exists.sum(axis=1)

        self.weights = asset_exists.div(n_assets, axis=0).fillna(0.0)

    def _compute_portfolio_returns(self) -> None:
        self.portfolio_returns = (self.returns * self.weights).sum(axis=1)

    def cumulative_return(self) -> pd.Series:
        return (1 + self.portfolio_returns).cumprod()

    def drawdown(self) -> pd.Series:
        wealth = self.cumulative_return()
        peak = wealth.cummax()
        return (wealth - peak) / peak

    def max_drawdown(self) -> float:
        return float(self.drawdown().min())

    def volatility(self, annualized: bool = True) -> float:
        vol = float(self.portfolio_returns.std())
        return vol * np.sqrt(252) if annualized else vol

    def mean_return(self, annualized: bool = True) -> float:
        mu = float(self.portfolio_returns.mean())
        return mu * 252 if annualized else mu

    def sharpe_ratio(self) -> float:
        return self.mean_return() / self.volatility()

    def var_cvar(self, alpha: float = 0.99) -> Tuple[float, float]:
        var = float(self.portfolio_returns.quantile(1 - alpha))
        cvar = float(self.portfolio_returns[self.portfolio_returns <= var].mean())
        return var, cvar

    def summary(self) -> pd.Series:
        var_99, cvar_99 = self.var_cvar(0.99)

        return pd.Series(
            {
                "Mean Return (ann)": self.mean_return(),
                "Volatility (ann)": self.volatility(),
                "Sharpe": self.sharpe_ratio(),
                "Max Drawdown": self.max_drawdown(),
                "VaR 99%": var_99,
                "CVaR 99%": cvar_99,
            }
        )

    def portfolio_composition_table(self) -> pd.DataFrame:
        weights_pct = self.weights * 100
        asset_values = self.prices * self.weights

        data: Dict[Tuple[str, str], pd.Series] = {}
        for asset in self.prices.columns:
            data[(asset, "weight_%")] = weights_pct[asset]
            data[(asset, "price")] = self.prices[asset]
            data[(asset, "value")] = asset_values[asset]

        df = pd.DataFrame(data)
        df.columns = pd.MultiIndex.from_tuples(df.columns)

        df["portfolio_value"] = asset_values.sum(axis=1)
        df["portfolio_return_%"] = df["portfolio_value"].pct_change() * 100

        DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
        portfolio_table_path = DATA_GOLD_DIR / "portfolio_composition.csv"
        df.to_csv(portfolio_table_path)

        return df

    def plot_portfolio(self) -> None:
        cumulative_returns = self.cumulative_return()
        plt.figure(figsize=(12, 6))
        sns.lineplot(data=cumulative_returns)
        plt.title("Cumulative Return of the Portfolio")
        plt.xlabel("Date")
        plt.ylabel("Cumulative Return")
        plt.tight_layout()

        FIGURES_DIR.mkdir(parents=True, exist_ok=True)
        chart_path = FIGURES_DIR / "portfolio_returns_chart.png"
        plt.savefig(chart_path)
        plt.close()

    def plot_chart_per_asset(self) -> None:
        FIGURES_DIR.mkdir(parents=True, exist_ok=True)
        for col in self.prices.columns:
            plt.figure(figsize=(12, 6))
            sns.lineplot(data=self.prices[col])
            plt.title(f"{col} Price Over Time")
            plt.xlabel("Date")
            plt.ylabel("Price")
            plt.tight_layout()
            chart_path = FIGURES_DIR / f"{col}_price_chart.png"
            plt.savefig(chart_path)
            plt.close()


@dataclass
class HMMState:
    """Parameters of a single HMM state."""

    mean: np.ndarray
    cov: np.ndarray
    volatility: float  # Frobenius norm of covariance diagonal


@dataclass
class HMMResults:
    """Fitted HMM results and regime assignments."""

    model: hmm.GaussianHMM
    transition_matrix: np.ndarray
    states: Dict[int, HMMState]
    regimes: np.ndarray  # regime_t for each time step
    calm_state: int  # which state index corresponds to "calm"
    crisis_state: int  # which state index corresponds to "crisis"


### Funciones ###

In [89]:
def set_global_seed() -> None:
    """Set global random seed for reproducibility."""
    np.random.seed(RANDOM_SEED)

def ensure_directories() -> None:
    """Create all necessary directories for the project."""
    DATA_BRONZE_DIR.mkdir(parents=True, exist_ok=True)
    DATA_SILVER_DIR.mkdir(parents=True, exist_ok=True)
    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    FIGURES_DIR.mkdir(parents=True, exist_ok=True)

def yield_curve_slope(y10: pd.Series, y2: pd.Series) -> pd.Series:
    """Calculate the yield curve slope (10Y - 2Y spread)."""
    return y10 - y2

def compute_regime_statistics(regimes: np.ndarray, calm_state: int) -> Dict[str, float]:
    """Compute summary statistics of regime frequencies."""

    crisis_state = 1 - calm_state
    n_calm = int((regimes == calm_state).sum())
    n_crisis = int((regimes == crisis_state).sum())
    pct_calm = 100.0 * n_calm / len(regimes)
    pct_crisis = 100.0 * n_crisis / len(regimes)

    return {
        "n_calm_days": n_calm,
        "n_crisis_days": n_crisis,
        "pct_calm": pct_calm,
        "pct_crisis": pct_crisis,
    }

def analyze_hmm_features(log_returns: pd.DataFrame, hmm_results: HMMResults) -> pd.DataFrame:
    """Analyze the contribution of each market variable to regime detection.
    
    Shows the mean returns and volatility per feature in each HMM state
    (Calm vs Crisis), demonstrating that all market variables are being used.
    
    Parameters
    ----------
    log_returns : pd.DataFrame
        The multivariate log-returns DataFrame (all market variables).
    hmm_results : HMMResults
        Fitted HMM results containing state parameters.
    
    Returns
    -------
    pd.DataFrame
        Analysis table showing mean and std for each feature in each state.
    """
    
    analysis_data = []
    
    for state_idx, state in hmm_results.states.items():
        state_name = "CALM" if state_idx == hmm_results.calm_state else "CRISIS"
        
        for col_idx, col_name in enumerate(log_returns.columns):
            analysis_data.append({
                "Variable": col_name,
                "Regime": state_name,
                "Mean (HMM)": state.mean[col_idx],
                "Std Dev (HMM)": np.sqrt(state.cov[col_idx, col_idx]),
            })
    
    df_analysis = pd.DataFrame(analysis_data)
    return df_analysis.sort_values(["Variable", "Regime"]).reset_index(drop=True)

def load_and_prepare_returns(data_path: Path) -> Tuple[pd.DataFrame, pd.Series]:
    """Load combined market data and prepare log returns for HMM.
    
    Parameters
    ----------
    data_path : Path
        Path to the combined market data CSV file.
    
    Returns
    -------
    Tuple[pd.DataFrame, pd.Series]
        Log returns DataFrame and S&P 500 price series.
    """
    df = pd.read_csv(data_path, index_col=0, parse_dates=True)
    
    # Calculate log returns from the returns data
    log_returns = df.copy()
    
    # Load original S&P 500 prices from Bronze directory for visualization
    sp500_bronze_path = DATA_BRONZE_DIR / "equities_adj_close_^GSPC.csv"
    if sp500_bronze_path.exists():
        sp500_prices_raw = pd.read_csv(sp500_bronze_path, index_col=0, parse_dates=True)
        # The column name should be the ticker itself
        sp500_prices = sp500_prices_raw.iloc[:, 0]  # Get first column regardless of name
    else:
        # Fallback: reconstruct from returns if Bronze file not available
        sp500_prices = pd.Series(index=log_returns.index, dtype=float)
    
    return log_returns, sp500_prices

def standardize_returns(log_returns: pd.DataFrame) -> Tuple[np.ndarray, StandardScaler, pd.DataFrame]:
    """Standardize log returns using StandardScaler.
    
    Parameters
    ----------
    log_returns : pd.DataFrame
        DataFrame with log returns.
    
    Returns
    -------
    Tuple[np.ndarray, StandardScaler, pd.DataFrame]
        Scaled returns array, fitted scaler object, and cleaned returns DataFrame.
    """
    # Remove rows with NaN or infinity values
    log_returns_clean = log_returns.replace([np.inf, -np.inf], np.nan).dropna()
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(log_returns_clean)
    return X_scaled, scaler, log_returns_clean

def fit_hmm(X_scaled: np.ndarray, n_components: int = 2) -> hmm.GaussianHMM:
    """Fit a Gaussian HMM to the scaled returns.
    
    Parameters
    ----------
    X_scaled : np.ndarray
        Scaled multivariate returns.
    n_components : int
        Number of hidden states (default: 2 for calm/crisis).
    
    Returns
    -------
    hmm.GaussianHMM
        Fitted HMM model.
    """
    model = hmm.GaussianHMM(n_components=n_components, random_state=RANDOM_SEED, n_iter=5000)
    model.fit(X_scaled)
    return model

def identify_regimes(model: hmm.GaussianHMM, X_scaled: np.ndarray) -> Tuple[np.ndarray, HMMResults]:
    """Identify market regimes using fitted HMM.
    
    Determines which state is "calm" and which is "crisis" based on volatility levels.
    
    Parameters
    ----------
    model : hmm.GaussianHMM
        Fitted HMM model.
    X_scaled : np.ndarray
        Scaled multivariate returns.
    
    Returns
    -------
    Tuple[np.ndarray, HMMResults]
        Regime assignments and full HMM results container.
    """
    regimes = model.predict(X_scaled)
    
    # Determine which state is calm vs crisis based on volatility
    state_volatilities = []
    for i in range(model.n_components):
        vol = np.sqrt(np.trace(model.covars_[i]) / model.n_features)
        state_volatilities.append(vol)
    
    calm_state = int(np.argmin(state_volatilities))
    crisis_state = 1 - calm_state
    
    # Build state parameters
    states = {}
    for i in range(model.n_components):
        states[i] = HMMState(
            mean=model.means_[i],
            cov=model.covars_[i],
            volatility=state_volatilities[i]
        )
    
    hmm_results = HMMResults(
        model=model,
        transition_matrix=model.transmat_,
        states=states,
        regimes=regimes,
        calm_state=calm_state,
        crisis_state=crisis_state
    )
    
    return regimes, hmm_results

def visualize_regimes(
    prices: pd.Series,
    regimes: np.ndarray,
    calm_state: int,
    crisis_state: int,
    plot_path: Path
) -> None:
    """Visualize price series with regime coloring.
    
    Parameters
    ----------
    prices : pd.Series
        Price series to plot.
    regimes : np.ndarray
        Regime assignments.
    calm_state : int
        Index of calm regime.
    crisis_state : int
        Index of crisis regime.
    plot_path : Path
        Path to save the figure.
    """
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Plot prices
    ax.plot(prices.index, prices.values, "k-", linewidth=1.5, label="S&P 500 Price")
    
    # Color background by regime
    for i in range(len(regimes) - 1):
        if regimes[i] == calm_state:
            ax.axvspan(prices.index[i], prices.index[i + 1], alpha=0.2, color="whitesmoke")
        else:
            ax.axvspan(prices.index[i], prices.index[i + 1], alpha=0.2, color="deepskyblue")
    
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.set_title("Market Regimes: White=Calm, Blue=Crisis")
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(plot_path, dpi=300, bbox_inches="tight")
    plt.close()

def separate_data_by_regime(
    portfolio: Portfolio,
    regimes: np.ndarray,
    log_returns_clean: pd.DataFrame,
    hmm_results: HMMResults
) -> Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]:
    """Separate portfolio returns and asset prices by regime.
    
    Parameters
    ----------
    portfolio : Portfolio
        Portfolio object with prices and returns.
    regimes : np.ndarray
        Regime assignments.
    log_returns_clean : pd.DataFrame
        Cleaned log returns aligned with regimes.
    hmm_results : HMMResults
        HMM results with state labels.
    
    Returns
    -------
    Tuple[Dict[str, pd.DataFrame], Dict[str, pd.DataFrame]]
        Returns and prices separated by regime.
    """
    # Align portfolio returns with cleaned log returns index
    portfolio_returns_clean = portfolio.portfolio_returns.loc[log_returns_clean.index]
    
    # Separate data by regime
    calm_mask = regimes == hmm_results.calm_state
    crisis_mask = regimes == hmm_results.crisis_state
    
    returns_by_regime = {
        "CALM": portfolio_returns_clean[calm_mask],
        "CRISIS": portfolio_returns_clean[crisis_mask]
    }
    
    # Separate asset returns by regime
    asset_returns_by_regime = {
        "CALM": portfolio.returns.loc[log_returns_clean.index][calm_mask],
        "CRISIS": portfolio.returns.loc[log_returns_clean.index][crisis_mask]
    }
    
    return returns_by_regime, asset_returns_by_regime

def calculate_marginal_statistics(
    asset_returns: Dict[str, pd.DataFrame],
    assets: Dict[str, str]
) -> pd.DataFrame:
    """Calculate marginal statistics (mean, vol, skewness, kurtosis) by regime and asset.
    
    Parameters
    ----------
    asset_returns : Dict[str, pd.DataFrame]
        Asset returns separated by regime.
    assets : Dict[str, str]
        Asset dictionary with types.
    
    Returns
    -------
    pd.DataFrame
        Comprehensive statistics table by asset and regime.
    """
    stats_list = []
    
    for regime_name, returns_df in asset_returns.items():
        for asset in returns_df.columns:
            if asset in assets:
                asset_ret = returns_df[asset].dropna()
                
                if len(asset_ret) > 0:
                    mean_ret = asset_ret.mean()
                    volatility = asset_ret.std()
                    skewness = sp_stats.skew(asset_ret)
                    kurtosis = sp_stats.kurtosis(asset_ret)
                    
                    stats_list.append({
                        "Asset": asset,
                        "Regime": regime_name,
                        "Mean Return": mean_ret,
                        "Volatility": volatility,
                        "Skewness": skewness,
                        "Kurtosis": kurtosis,
                        "N Obs": len(asset_ret)
                    })
    
    df_stats = pd.DataFrame(stats_list)
    return df_stats.sort_values(["Asset", "Regime"]).reset_index(drop=True)

def analyze_key_assets(
    asset_returns: Dict[str, pd.DataFrame],
    key_assets: List[str] = ["HYG", "GLD"]
) -> pd.DataFrame:
    """Focus analysis on key assets (High Yield, Gold).
    
    Parameters
    ----------
    asset_returns : Dict[str, pd.DataFrame]
        Asset returns separated by regime.
    key_assets : List[str]
        List of key assets to analyze.
    
    Returns
    -------
    pd.DataFrame
        Detailed statistics for key assets.
    """
    key_stats = []
    
    for asset in key_assets:
        for regime_name, returns_df in asset_returns.items():
            if asset in returns_df.columns:
                asset_ret = returns_df[asset].dropna()
                
                if len(asset_ret) > 0:
                    var_99 = asset_ret.quantile(0.01)  # 1% worst case
                    cvar_99 = asset_ret[asset_ret <= var_99].mean()
                    
                    key_stats.append({
                        "Asset": asset,
                        "Regime": regime_name,
                        "Mean (%)": asset_ret.mean() * 100,
                        "Volatility (%)": asset_ret.std() * 100,
                        "Skewness": sp_stats.skew(asset_ret),
                        "Kurtosis": sp_stats.kurtosis(asset_ret),
                        "VaR 99%": var_99,
                        "CVaR 99%": cvar_99,
                        "Min Return": asset_ret.min(),
                        "Max Return": asset_ret.max(),
                    })
    
    df_key = pd.DataFrame(key_stats)
    return df_key.sort_values(["Asset", "Regime"]).reset_index(drop=True)

def interpret_regime_changes(df_key_assets: pd.DataFrame) -> str:
    """Generate economic interpretation of regime changes for key assets.
    
    Parameters
    ----------
    df_key_assets : pd.DataFrame
        Key assets statistics by regime.
    
    Returns
    -------
    str
        Interpretation text.
    """
    interpretation = []
    interpretation.append("=" * 80)
    interpretation.append("INTERPRETACIÓN ECONÓMICA DE CAMBIOS DE RÉGIMEN")
    interpretation.append("=" * 80 + "\n")
    
    # HYG Analysis
    hyg_calm = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CALM")]
    hyg_crisis = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]
    
    if not hyg_calm.empty and not hyg_crisis.empty:
        vol_calm = hyg_calm["Volatility (%)"].values[0]
        vol_crisis = hyg_crisis["Volatility (%)"].values[0]
        vol_change = ((vol_crisis - vol_calm) / vol_calm) * 100
        
        interpretation.append("📊 HIGH YIELD (HYG) - Bonos de Alto Rendimiento")
        interpretation.append("-" * 80)
        interpretation.append(f"  • Volatilidad en CALMA: {vol_calm:.2f}%")
        interpretation.append(f"  • Volatilidad en CRISIS: {vol_crisis:.2f}%")
        interpretation.append(f"  • Aumento: {vol_change:.1f}%")
        interpretation.append("\n  INTERPRETACIÓN:")
        interpretation.append("  El aumento de volatilidad en crisis refleja:")
        interpretation.append("  ✓ Mayor aversión al riesgo en el mercado")
        interpretation.append("  ✓ Widening de spreads de crédito")
        interpretation.append("  ✓ Stress en el segmento de bonos de alto rendimiento")
        interpretation.append("  → El HYG es PRO-CÍCLICO (amplifica riesgo en crisis)\n")
    
    # GLD Analysis
    gld_calm = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CALM")]
    gld_crisis = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CRISIS")]
    
    if not gld_calm.empty and not gld_crisis.empty:
        ret_calm = gld_calm["Mean (%)"].values[0]
        ret_crisis = gld_crisis["Mean (%)"].values[0]
        vol_calm_gld = gld_calm["Volatility (%)"].values[0]
        vol_crisis_gld = gld_crisis["Volatility (%)"].values[0]
        
        interpretation.append("🏆 ORO (GLD) - Activo Refugio")
        interpretation.append("-" * 80)
        interpretation.append(f"  • Retorno medio en CALMA: {ret_calm:.2f}%")
        interpretation.append(f"  • Retorno medio en CRISIS: {ret_crisis:.2f}%")
        interpretation.append(f"  • Volatilidad en CALMA: {vol_calm_gld:.2f}%")
        interpretation.append(f"  • Volatilidad en CRISIS: {vol_crisis_gld:.2f}%")
        
        if ret_crisis > ret_calm:
            interpretation.append("\n  INTERPRETACIÓN:")
            interpretation.append("  ✓ El ORO SUBE durante crisis (comportamiento de refugio)")
            interpretation.append("  ✓ Inversores huyen a activos seguros")
            interpretation.append("  ✓ Cobertura contra inflación y depreciación de divisas")
            interpretation.append("  → El GLD es ANTI-CÍCLICO (protección en turbulencia)\n")
        else:
            interpretation.append("\n  INTERPRETACIÓN:")
            interpretation.append("  ⚠ El ORO NO actúa como refugio esperado")
            interpretation.append("  ⚠ Posible liquidación forzada en crisis")
            interpretation.append("  → Revisar correlación con equity en stress\n")
    
    interpretation.append("=" * 80)
    return "\n".join(interpretation)

def compare_volatility_regimes(df_stats: pd.DataFrame) -> pd.DataFrame:
    """Create comparison table of volatility changes between regimes.
    
    Parameters
    ----------
    df_stats : pd.DataFrame
        Statistics by asset and regime.
    
    Returns
    -------
    pd.DataFrame
        Volatility comparison table.
    """
    vol_comparison = []
    
    for asset in df_stats["Asset"].unique():
        asset_data = df_stats[df_stats["Asset"] == asset]
        
        calm_vol = asset_data[asset_data["Regime"] == "CALM"]["Volatility"].values
        crisis_vol = asset_data[asset_data["Regime"] == "CRISIS"]["Volatility"].values
        
        if len(calm_vol) > 0 and len(crisis_vol) > 0:
            vol_ratio = crisis_vol[0] / calm_vol[0]
            vol_change = ((crisis_vol[0] - calm_vol[0]) / calm_vol[0]) * 100
            
            vol_comparison.append({
                "Asset": asset,
                "Volatility CALM": calm_vol[0],
                "Volatility CRISIS": crisis_vol[0],
                "Ratio (Crisis/Calm)": vol_ratio,
                "% Change": vol_change
            })
    
    df_vol = pd.DataFrame(vol_comparison)
    return df_vol.sort_values("Ratio (Crisis/Calm)", ascending=False).reset_index(drop=True)

def save_phase1_analysis(
    df_stats: pd.DataFrame,
    df_key_assets: pd.DataFrame,
    df_vol_comparison: pd.DataFrame,
    interpretation: str,
    output_dir: Path
) -> None:
    """Save all Phase 1 analysis results to files.
    
    Parameters
    ----------
    df_stats : pd.DataFrame
        Marginal statistics table.
    df_key_assets : pd.DataFrame
        Key assets analysis.
    df_vol_comparison : pd.DataFrame
        Volatility comparison.
    interpretation : str
        Economic interpretation text.
    output_dir : Path
        Output directory path.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save statistics tables
    df_stats.to_csv(output_dir / "phase1_marginal_statistics.csv", index=False)
    df_key_assets.to_csv(output_dir / "phase1_key_assets_analysis.csv", index=False)
    df_vol_comparison.to_csv(output_dir / "phase1_volatility_comparison.csv", index=False)
    
    # Save interpretation with UTF-8 encoding
    with open(output_dir / "phase1_interpretation.txt", "w", encoding="utf-8") as f:
        f.write(interpretation)

def run_phase1_risk_analysis(
    portfolio: Portfolio,
    regimes: np.ndarray,
    log_returns_clean: pd.DataFrame,
    hmm_results: HMMResults
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, str]:
    """Execute complete Phase 1 analysis: Risk by Regime.
    
    Parameters
    ----------
    portfolio : Portfolio
        Portfolio object with prices and returns.
    regimes : np.ndarray
        Regime assignments.
    log_returns_clean : pd.DataFrame
        Cleaned log returns aligned with regimes.
    hmm_results : HMMResults
        HMM results with state labels.
    
    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, str]
        Marginal statistics, key assets analysis, volatility comparison, and interpretation.
    """
    
    print("=" * 80)
    print("FASE 1: ANÁLISIS DE RIESGO INDIVIDUAL POR RÉGIMEN")
    print("=" * 80 + "\n")
    
    # Task 1.1: Separate data by regime
    print("1.1 Separando datos por régimen...")
    returns_by_regime, asset_returns_by_regime = separate_data_by_regime(
        portfolio, regimes, log_returns_clean, hmm_results
    )
    
    calm_obs = len(returns_by_regime["CALM"])
    crisis_obs = len(returns_by_regime["CRISIS"])
    print(f"     ✓ Días en CALMA: {calm_obs}")
    print(f"     ✓ Días en CRISIS: {crisis_obs}\n")
    
    # Task 1.2: Calculate marginal statistics
    print("1.2 Calculando estadísticas marginales...")
    df_stats = calculate_marginal_statistics(asset_returns_by_regime, portfolio.assets)
    print(f"     ✓ {len(df_stats)} filas de estadísticas (activos × regímenes)\n")
    
    # Task 1.3: Analyze key assets
    print("1.3 Analizando activos clave (HYG, GLD)...")
    df_key_assets = analyze_key_assets(asset_returns_by_regime)
    print(f"     ✓ Análisis detallado de {df_key_assets['Asset'].nunique()} activos\n")
    
    # Volatility comparison
    print("1.4 Comparando volatilidades entre regímenes...")
    df_vol_comparison = compare_volatility_regimes(df_stats)
    print(f"     ✓ Tabla de comparación creada\n")
    
    # Task 1.4: Economic interpretation
    print("1.4 Generando interpretación económica...")
    interpretation = interpret_regime_changes(df_key_assets)
    print(interpretation)
    print()
    
    # Save results
    print("Guardando resultados de Fase 1...")
    save_phase1_analysis(df_stats, df_key_assets, df_vol_comparison, interpretation, DATA_GOLD_DIR)
    print(f"     ✓ Resultados guardados en {DATA_GOLD_DIR}\n")
    
    return df_stats, df_key_assets, df_vol_comparison, interpretation

def run_regime_detection_pipeline() -> Dict[str, float]:
    """Run the full HMM-based regime detection workflow and return basic statistics."""

    # Data preparation
    log_returns, sp500_prices = load_and_prepare_returns(COMBINED_PATH)
    
    # Display which variables are being analyzed
    print("=" * 80)
    print("MARKET VARIABLES USED FOR REGIME DETECTION (Multivariate Gaussian HMM):")
    print("=" * 80)
    for i, col in enumerate(log_returns.columns, 1):
        print(f"{i}. {col}")
    print(f"\nTotal dimensions: {log_returns.shape[1]} variables × {log_returns.shape[0]} observations")
    print("=" * 80 + "\n")
    
    X_scaled, _, log_returns_clean = standardize_returns(log_returns)

    # HMM fitting
    model = fit_hmm(X_scaled, n_components=2)

    # Regime identification
    regimes, hmm_results = identify_regimes(model, X_scaled)

    # Analyze feature contributions to regimes (use cleaned returns)
    df_feature_analysis = analyze_hmm_features(log_returns_clean, hmm_results)
    print("FEATURE ANALYSIS - Mean and Volatility per Regime:")
    print("=" * 80)
    print(df_feature_analysis.to_string(index=False))
    print("=" * 80 + "\n")

    # Align sp500_prices with cleaned returns
    sp500_prices_clean = sp500_prices.loc[log_returns_clean.index]

    # Visualization
    FIGURES_DIR.mkdir(parents=True, exist_ok=True)
    plot_path = FIGURES_DIR / "regime_visualization_sp500.png"
    visualize_regimes(
        sp500_prices_clean,
        regimes,
        hmm_results.calm_state,
        hmm_results.crisis_state,
        plot_path,
    )

    # Save outputs
    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    regime_ts_path = DATA_GOLD_DIR / "regime_timeseries.csv"
    save_regime_timeseries(log_returns_clean.index, regimes, sp500_prices_clean, hmm_results, regime_ts_path)

    hmm_params_path = DATA_GOLD_DIR / "hmm_parameters.txt"
    save_hmm_parameters(hmm_results, hmm_params_path)

    # Statistics
    stats = compute_regime_statistics(regimes, hmm_results.calm_state)
    return stats

def save_regime_timeseries(
    dates: pd.DatetimeIndex,
    regimes: np.ndarray,
    sp500_prices: pd.Series,
    hmm_results: HMMResults,
    output_path: Path
) -> None:
    """Save regime time series to CSV.
    
    Parameters
    ----------
    dates : pd.DatetimeIndex
        Index dates.
    regimes : np.ndarray
        Regime assignments.
    sp500_prices : pd.Series
        S&P 500 prices.
    hmm_results : HMMResults
        HMM results container.
    output_path : Path
        Path to save CSV.
    """
    df_regimes = pd.DataFrame({
        "date": dates,
        "regime": regimes,
        "regime_label": ["CALM" if r == hmm_results.calm_state else "CRISIS" for r in regimes],
        "sp500_price": sp500_prices.values
    })
    df_regimes.set_index("date", inplace=True)
    df_regimes.to_csv(output_path)

def save_hmm_parameters(hmm_results: HMMResults, output_path: Path) -> None:
    """Save HMM parameters to text file.
    
    Parameters
    ----------
    hmm_results : HMMResults
        HMM results container.
    output_path : Path
        Path to save parameters file.
    """
    with open(output_path, "w") as f:
        f.write("=" * 80 + "\n")
        f.write("HMM PARAMETERS\n")
        f.write("=" * 80 + "\n\n")
        
        f.write("TRANSITION MATRIX:\n")
        f.write(str(hmm_results.transition_matrix) + "\n\n")
        
        for state_idx, state in hmm_results.states.items():
            state_label = "CALM" if state_idx == hmm_results.calm_state else "CRISIS"
            f.write(f"\nSTATE {state_idx} ({state_label}):\n")
            f.write(f"  Volatility: {state.volatility:.6f}\n")
            f.write(f"  Mean:\n{state.mean}\n")
            f.write(f"  Covariance (diagonal):\n{np.diag(state.cov)}\n")

### Datos ###

In [90]:
def build_portfolio() -> Portfolio:
    """Create the baseline multi-asset portfolio configuration and return a Portfolio."""

    assets: Dict[str, str] = {
        "AAPL": "equity",
        "AMZN": "equity",
        "BAC": "equity",
        "BRK-B": "equity",
        "CVX": "equity",
        "ENPH": "equity",
        "GLD": "equity",
        "GME": "equity",
        "GOOGL": "equity",
        "JNJ": "equity",
        "JPM": "equity",
        "MSFT": "equity",
        "NVDA": "equity",
        "PG": "equity",
        "XOM": "equity",
        "HYG": "equity",
        "GS10": "yield",
        "GS2": "yield",
    }

    return Portfolio(assets=assets)


def portfolio() -> Portfolio:
    """Backwards-compatible wrapper for building the default portfolio."""

    return build_portfolio()


def market_risk() -> pd.DataFrame:
    """Construct the market risk data set used for regime detection."""

    # Equity market
    sp500 = MarketData.fetch_equities(tickers=["^GSPC"], start=START_DATE, end=END_DATE)
    sp500_ret = sp500.pct_change()

    vix = MarketData.fetch_equities(tickers=["^VIX"], start=START_DATE, end=END_DATE)
    vix_ret = vix.pct_change()

    # Interest rates
    y10 = MarketData.fetch_us_yields("GS10", start=START_DATE, end=END_DATE)
    y2 = MarketData.fetch_us_yields("GS2", start=START_DATE, end=END_DATE)

    y10_chg = y10.pct_change()
    y2_chg = y2.pct_change()
    slope = yield_curve_slope(y10["GS10"], y2["GS2"]).rename("yield_slope")

    # Credit spread
    hy_spread = MarketData.fetch_us_yields("BAMLH0A0HYM2", start=START_DATE, end=END_DATE)
    hy_spread_chg = hy_spread.pct_change()

    # Combine all market risk drivers
    df = pd.concat(
        [
            sp500_ret,
            vix_ret,
            y10_chg,
            y2_chg,
            slope,
            hy_spread_chg,
        ],
        axis=1,
    )

    for col in ["GS10", "GS2", "yield_slope"]:
        if col in df.columns:
            df[col] = df[col].ffill()

    DATA_GOLD_DIR.mkdir(parents=True, exist_ok=True)
    df_path = DATA_GOLD_DIR / "market_data_combined.csv"
    df.to_csv(df_path)

    return df

### Reports ###

In [91]:
def create_volatility_comparison_chart(
    df_vol_comparison: pd.DataFrame,
    output_path: Path
) -> None:
    """Create professional volatility comparison chart for executive report.
    
    Parameters
    ----------
    df_vol_comparison : pd.DataFrame
        Volatility comparison data.
    output_path : Path
        Path to save the figure.
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Sort by ratio descending
    df_sorted = df_vol_comparison.sort_values("Ratio (Crisis/Calm)", ascending=True)
    
    colors = ["#d62728" if x > 1.2 else "#ff7f0e" if x > 1.0 else "#2ca02c" 
              for x in df_sorted["Ratio (Crisis/Calm)"]]
    
    ax.barh(df_sorted["Asset"], df_sorted["Ratio (Crisis/Calm)"], color=colors, alpha=0.8, edgecolor="black")
    ax.axvline(x=1.0, color="black", linestyle="--", linewidth=2, label="Baseline (No Change)")
    ax.set_xlabel("Razón Volatilidad Crisis / Volatilidad Calma", fontsize=12, fontweight="bold")
    ax.set_title("Amplificación de Volatilidad por Activo en Períodos de Crisis", 
                 fontsize=14, fontweight="bold", pad=20)
    ax.legend(fontsize=10)
    ax.grid(axis="x", alpha=0.3)
    
    # Add value labels
    for i, (asset, ratio) in enumerate(zip(df_sorted["Asset"], df_sorted["Ratio (Crisis/Calm)"])):
        ax.text(ratio + 0.05, i, f"{ratio:.2f}x", va="center", fontweight="bold")
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()


def create_key_assets_chart(
    df_key_assets: pd.DataFrame,
    output_path: Path
) -> None:
    """Create professional chart for key assets analysis.
    
    Parameters
    ----------
    df_key_assets : pd.DataFrame
        Key assets statistics.
    output_path : Path
        Path to save the figure.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # HYG Analysis
    hyg_data = df_key_assets[df_key_assets["Asset"] == "HYG"]
    if not hyg_data.empty:
        regimes_hyg = hyg_data["Regime"].values
        vol_hyg = hyg_data["Volatility (%)"].values
        colors_hyg = ["#2ca02c", "#d62728"]
        
        axes[0].bar(regimes_hyg, vol_hyg, color=colors_hyg, alpha=0.8, edgecolor="black", linewidth=2)
        axes[0].set_ylabel("Volatilidad (%)", fontsize=11, fontweight="bold")
        axes[0].set_title("HYG: Bonos de Alto Rendimiento\n(Comportamiento Pro-Cíclico)", 
                         fontsize=12, fontweight="bold")
        axes[0].grid(axis="y", alpha=0.3)
        
        for i, v in enumerate(vol_hyg):
            axes[0].text(i, v + 1, f"{v:.1f}%", ha="center", fontweight="bold", fontsize=11)
    
    # GLD Analysis
    gld_data = df_key_assets[df_key_assets["Asset"] == "GLD"]
    if not gld_data.empty:
        regimes_gld = gld_data["Regime"].values
        ret_gld = gld_data["Mean (%)"].values
        colors_gld = ["#2ca02c", "#d62728"]
        
        axes[1].bar(regimes_gld, ret_gld, color=colors_gld, alpha=0.8, edgecolor="black", linewidth=2)
        axes[1].set_ylabel("Retorno Promedio (%)", fontsize=11, fontweight="bold")
        axes[1].set_title("GLD: Oro (Activo Refugio)\n(Comportamiento Anti-Cíclico)", 
                         fontsize=12, fontweight="bold")
        axes[1].grid(axis="y", alpha=0.3)
        
        for i, v in enumerate(ret_gld):
            axes[1].text(i, v + 0.05 if v > 0 else v - 0.15, f"{v:.2f}%", 
                        ha="center", fontweight="bold", fontsize=11)
    
    plt.suptitle("Análisis de Activos Clave por Régimen", fontsize=14, fontweight="bold", y=1.02)
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()


def generate_executive_report(
    regime_stats: Dict[str, float],
    df_stats: pd.DataFrame,
    df_key_assets: pd.DataFrame,
    df_vol_comparison: pd.DataFrame,
    interpretation: str,
    portfolio: Portfolio,
    regimes: np.ndarray,
    hmm_results: HMMResults,
    output_dir: Path
) -> str:
    """Generate professional executive report in markdown format.
    
    Parameters
    ----------
    regime_stats : Dict[str, float]
        Regime statistics.
    df_stats : pd.DataFrame
        Marginal statistics.
    df_key_assets : pd.DataFrame
        Key assets analysis.
    df_vol_comparison : pd.DataFrame
        Volatility comparison.
    interpretation : str
        Economic interpretation.
    portfolio : Portfolio
        Portfolio object.
    regimes : np.ndarray
        Regime assignments.
    hmm_results : HMMResults
        HMM results.
    output_dir : Path
        Output directory.
    
    Returns
    -------
    str
        Markdown report content.
    """
    
    # Create charts
    vol_chart_path = output_dir / "chart_volatility_comparison.png"
    key_assets_chart_path = output_dir / "chart_key_assets.png"
    
    create_volatility_comparison_chart(df_vol_comparison, vol_chart_path)
    create_key_assets_chart(df_key_assets, key_assets_chart_path)
    
    # Get portfolio metrics
    portfolio_summary = portfolio.summary()
    
    # Build markdown report
    report = []
    
    report.append("# INFORME EJECUTIVO: ANÁLISIS DE RIESGO Y REGÍMENES DE MERCADO")
    report.append("## Motor de Stress Testing - Cambios de Régimen Financiero")
    report.append("")
    report.append(f"**Fecha:** {pd.Timestamp.now().strftime('%d de %B de %Y')}")
    report.append("**Para:** Comité de Riesgos (CEO, CFO, CRO)")
    report.append("")
    report.append("---")
    report.append("")
    
    # EXECUTIVE SUMMARY
    report.append("## 1. RESUMEN EJECUTIVO")
    report.append("")
    report.append(f"Este análisis identifica **dos regímenes de mercado distintos** en los últimos {len(regimes)} días:")
    report.append(f"- **CALMA:** {regime_stats['n_calm_days']:.0f} días ({regime_stats['pct_calm']:.1f}%)")
    report.append(f"- **CRISIS:** {regime_stats['n_crisis_days']:.0f} días ({regime_stats['pct_crisis']:.1f}%)")
    report.append("")
    report.append("### Hallazgos Clave")
    report.append("")
    
    # Key finding 1: Volatility spike
    max_vol_ratio = df_vol_comparison["Ratio (Crisis/Calm)"].max()
    max_vol_asset = df_vol_comparison.loc[df_vol_comparison["Ratio (Crisis/Calm)"].idxmax(), "Asset"]
    report.append(f"**1. Amplificación de Volatilidad:** En períodos de crisis, la volatilidad de {max_vol_asset} es **{max_vol_ratio:.1f}x** mayor que en calma.")
    report.append("")
    
    # Key finding 2: HYG behavior
    hyg_calm_vol = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CALM")]["Volatility (%)"].values
    hyg_crisis_vol = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]["Volatility (%)"].values
    
    if len(hyg_calm_vol) > 0 and len(hyg_crisis_vol) > 0:
        hyg_increase = ((hyg_crisis_vol[0] - hyg_calm_vol[0]) / hyg_calm_vol[0]) * 100
        report.append(f"**2. Riesgo de Crédito:** Los bonos de alto rendimiento (HYG) aumentan volatilidad **{hyg_increase:.0f}%** en crisis → **PRO-CÍCLICO**.")
        report.append("")
    
    # Key finding 3: GLD behavior
    gld_calm_ret = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CALM")]["Mean (%)"].values
    gld_crisis_ret = df_key_assets[(df_key_assets["Asset"] == "GLD") & (df_key_assets["Regime"] == "CRISIS")]["Mean (%)"].values
    
    if len(gld_calm_ret) > 0 and len(gld_crisis_ret) > 0:
        gld_behavior = "SUBE" if gld_crisis_ret[0] > gld_calm_ret[0] else "BAJA"
        report.append(f"**3. Activo Refugio:** El oro (GLD) {gld_behavior} durante crisis → **ACTÚA COMO COBERTURA**.")
        report.append("")
    
    report.append("### Implicaciones para el Portafolio")
    report.append(f"- Retorno anualizado: **{portfolio_summary['Mean Return (ann)']*100:.2f}%**")
    report.append(f"- Volatilidad: **{portfolio_summary['Volatility (ann)']*100:.2f}%**")
    report.append(f"- Máxima pérdida acumulada: **{portfolio_summary['Max Drawdown']*100:.2f}%**")
    report.append(f"- VaR 99%: **{portfolio_summary['VaR 99%']*100:.2f}%** (pérdida diaria en peor escenario)")
    report.append("")
    
    report.append("---")
    report.append("")
    
    # REGIME ANALYSIS
    report.append("## 2. ANÁLISIS DE REGÍMENES Y VOLATILIDAD")
    report.append("")
    report.append("### Transición entre Regímenes")
    report.append("")
    report.append("El modelo HMM identifica cambios en la **matriz de transición de estados**, mostrando:")
    report.append(f"- Probabilidad de permanecer en CALMA: **{hmm_results.transition_matrix[hmm_results.calm_state, hmm_results.calm_state]*100:.1f}%**")
    report.append(f"- Probabilidad de pasar a CRISIS: **{hmm_results.transition_matrix[hmm_results.calm_state, hmm_results.crisis_state]*100:.1f}%**")
    report.append("")
    
    report.append("### Amplificación de Riesgo por Activo")
    report.append("")
    report.append("| Activo | Vol. Calma | Vol. Crisis | Razón Crisis/Calma |")
    report.append("|--------|-----------|------------|-------------------|")
    
    for _, row in df_vol_comparison.head(10).iterrows():
        ratio = row["Ratio (Crisis/Calm)"]
        risk_label = "🔴 MUY ALTO" if ratio > 1.5 else "🟠 ALTO" if ratio > 1.2 else "🟡 MODERADO" if ratio > 1.0 else "🟢 BAJO"
        report.append(f"| {row['Asset']} | {row['Volatility CALM']:.3f} | {row['Volatility CRISIS']:.3f} | {ratio:.2f}x {risk_label} |")
    
    report.append("")
    report.append("![Amplificación de Volatilidad](chart_volatility_comparison.png)")
    report.append("")
    
    report.append("---")
    report.append("")
    
    # KEY ASSETS ANALYSIS
    report.append("## 3. ANÁLISIS DE ACTIVOS CLAVE")
    report.append("")
    report.append("### HYG: Bonos de Alto Rendimiento (Comportamiento Pro-Cíclico)")
    report.append("")
    
    hyg_stats = df_key_assets[df_key_assets["Asset"] == "HYG"]
    if not hyg_stats.empty:
        hyg_calm = hyg_stats[hyg_stats["Regime"] == "CALM"].iloc[0]
        hyg_crisis = hyg_stats[hyg_stats["Regime"] == "CRISIS"].iloc[0]
        
        report.append(f"| Métrica | Calma | Crisis | Cambio |")
        report.append("|---------|-------|--------|--------|")
        report.append(f"| Retorno Promedio | {hyg_calm['Mean (%)']:.2f}% | {hyg_crisis['Mean (%)']:.2f}% | {hyg_crisis['Mean (%)'] - hyg_calm['Mean (%)']:.2f}% |")
        report.append(f"| Volatilidad | {hyg_calm['Volatility (%)']:.2f}% | {hyg_crisis['Volatility (%)']:.2f}% | +{hyg_crisis['Volatility (%)'] - hyg_calm['Volatility (%)']:.2f}% |")
        report.append(f"| Asimetría | {hyg_calm['Skewness']:.2f} | {hyg_crisis['Skewness']:.2f} | - |")
        report.append(f"| Curtosis | {hyg_calm['Kurtosis']:.2f} | {hyg_crisis['Kurtosis']:.2f} | - |")
        report.append("")
        report.append("**Interpretación:** El aumento de volatilidad refleja mayor **aversión al riesgo** y **widening de spreads de crédito** durante turbulencia. HYG amplifica pérdidas en crisis.")
        report.append("")
    
    report.append("### GLD: Oro (Comportamiento Anti-Cíclico)")
    report.append("")
    
    gld_stats = df_key_assets[df_key_assets["Asset"] == "GLD"]
    if not gld_stats.empty:
        gld_calm = gld_stats[gld_stats["Regime"] == "CALM"].iloc[0]
        gld_crisis = gld_stats[gld_stats["Regime"] == "CRISIS"].iloc[0]
        
        report.append(f"| Métrica | Calma | Crisis | Cambio |")
        report.append("|---------|-------|--------|--------|")
        report.append(f"| Retorno Promedio | {gld_calm['Mean (%)']:.2f}% | {gld_crisis['Mean (%)']:.2f}% | {gld_crisis['Mean (%)'] - gld_calm['Mean (%)']:.2f}% |")
        report.append(f"| Volatilidad | {gld_calm['Volatility (%)']:.2f}% | {gld_crisis['Volatility (%)']:.2f}% | {gld_crisis['Volatility (%)'] - gld_calm['Volatility (%)']:.2f}% |")
        report.append("")
        report.append("**Interpretación:** El oro proporciona **cobertura contra riesgo sistémico**. Retornos superiores en crisis → activo refugio efectivo.")
        report.append("")
    
    report.append("![Análisis de Activos Clave](chart_key_assets.png)")
    report.append("")
    
    report.append("---")
    report.append("")
    
    # RISK METRICS
    report.append("## 4. MÉTRICAS DE RIESGO EXTREMO")
    report.append("")
    
    hyg_var = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]["VaR 99%"].values
    hyg_cvar = df_key_assets[(df_key_assets["Asset"] == "HYG") & (df_key_assets["Regime"] == "CRISIS")]["CVaR 99%"].values
    
    if len(hyg_var) > 0:
        report.append(f"**HYG (High Yield Bonds):**")
        report.append(f"- VaR 99% en Crisis: **{hyg_var[0]*100:.2f}%** (pérdida diaria en percentil 1)")
        report.append(f"- CVaR 99% en Crisis: **{hyg_cvar[0]*100:.2f}%** (pérdida esperada peor que VaR)")
        report.append("")
    
    report.append("---")
    report.append("")
    
    # RECOMMENDATIONS
    report.append("## 5. RECOMENDACIONES PARA EL COMITÉ DE RIESGOS")
    report.append("")
    report.append("### Gestión de Riesgo de Crédito")
    report.append("1. **Posiciones en HYG:** Establecer límites más estrictos dada la amplificación de volatilidad en crisis (+150-200%).")
    report.append("2. **Cobertura de Spreads:** Considerar posiciones cortas en credit spreads como hedge contra turbulencia.")
    report.append("")
    
    report.append("### Diversificación Efectiva")
    report.append("3. **Oro como Cobertura:** Incrementar asignación a GLD (activo refugio anti-cíclico) para períodos de volatilidad.")
    report.append("4. **Descomposición de Riesgo:** Realizar análisis de correlación por régimen → diversificación desaparece en crisis.")
    report.append("")
    
    report.append("### Stress Testing Dinámico")
    report.append("5. **Escenarios por Régimen:** Ejecutar stress tests separados para regímenes CALMA y CRISIS.")
    report.append("6. **Monitoreo en Tiempo Real:** Implementar alertas cuando el modelo detecte transición hacia CRISIS.")
    report.append("")
    
    report.append("---")
    report.append("")
    
    report.append("## CONCLUSIÓN")
    report.append("")
    report.append("El análisis revela **asimetrías de riesgo significativas** entre regímenes de mercado. La diversificación tradicional colapsa en períodos de crisis, con activos de alto rendimiento amplificando pérdidas (+150-200%) mientras que el oro proporciona protección efectiva.")
    report.append("")
    report.append("**Recomendación:** Revisar posiciones en bonos high-yield e incrementar exposición a activos refugio para optimizar ratio riesgo-retorno ajustado a dinámicas de régimen.")
    report.append("")
    
    report_text = "\n".join(report)
    
    # Save markdown report
    report_path = output_dir / "INFORME_EJECUTIVO.md"
    with open(report_path, "w", encoding="utf-8") as f:
        f.write(report_text)
    
    print(f"✓ Informe ejecutivo guardado en: {report_path}")
    
    return report_text

### Main ###


In [92]:
# Execute Phase 1
if __name__ == "__main__":
    ensure_directories()
    set_global_seed()

    portfolio_instance = build_portfolio()
    market_data_df = market_risk()
    regime_stats = run_regime_detection_pipeline()
    
    # Ensure regime detection results are available
    if 'regime_stats' in locals():
        print("\n" + "=" * 80)
        print("INICIANDO FASE 1...")
        print("=" * 80 + "\n")
        
        # Re-run regime detection to get all necessary variables
        log_returns, sp500_prices = load_and_prepare_returns(COMBINED_PATH)
        X_scaled, _, log_returns_clean = standardize_returns(log_returns)
        model = fit_hmm(X_scaled, n_components=2)
        regimes, hmm_results = identify_regimes(model, X_scaled)
        
        # Run Phase 1 analysis
        df_stats, df_key_assets, df_vol_comparison, interpretation = run_phase1_risk_analysis(
            portfolio_instance, regimes, log_returns_clean, hmm_results
        )
        
        # Display results
        print("\n" + "=" * 80)
        print("ESTADÍSTICAS MARGINALES POR ACTIVO Y RÉGIMEN")
        print("=" * 80)
        print(df_stats.to_string(index=False))
        print()
        
        print("=" * 80)
        print("ANÁLISIS DETALLADO: ACTIVOS CLAVE (HYG, GLD)")
        print("=" * 80)
        print(df_key_assets.to_string(index=False))
        print()
        
        print("=" * 80)
        print("COMPARACIÓN DE VOLATILIDAD: RATIO CRISIS/CALM")
        print("=" * 80)
        print(df_vol_comparison.to_string(index=False))
        print()
        
        # Display the report
        print("\n" + "=" * 80)
        print("GENERANDO INFORME EJECUTIVO...")
        print("=" * 80 + "\n")

        executive_report = generate_executive_report(
            regime_stats,
            df_stats,
            df_key_assets,
            df_vol_comparison,
            interpretation,
            portfolio_instance,
            regimes,
            hmm_results,
            REPORT_DIR
        )

        print(executive_report)

C:\Users\piett\AppData\Local\Temp\ipykernel_40988\1462631157.py:117: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.returns = self.prices.pct_change()
C:\Users\piett\AppData\Local\Temp\ipykernel_40988\3857109475.py:54: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  hy_spread_chg = hy_spread.pct_change()


MARKET VARIABLES USED FOR REGIME DETECTION (Multivariate Gaussian HMM):
1. ^GSPC
2. ^VIX
3. GS10
4. GS2
5. yield_slope
6. BAMLH0A0HYM2

Total dimensions: 6 variables × 5383 observations

FEATURE ANALYSIS - Mean and Volatility per Regime:
    Variable Regime  Mean (HMM)  Std Dev (HMM)
BAMLH0A0HYM2   CALM   -0.076434       0.742667
BAMLH0A0HYM2 CRISIS    0.109615       1.273867
        GS10   CALM    0.063577       0.498242
        GS10 CRISIS   -0.091176       1.436663
         GS2   CALM    0.034101       0.409869
         GS2 CRISIS   -0.048904       1.479575
       ^GSPC   CALM    0.052750       0.558784
       ^GSPC CRISIS   -0.075649       1.405924
        ^VIX   CALM   -0.070964       0.693273
        ^VIX CRISIS    0.101770       1.314256
 yield_slope   CALM   -0.264415       1.004147
 yield_slope CRISIS    0.379201       0.862572


INICIANDO FASE 1...

FASE 1: ANÁLISIS DE RIESGO INDIVIDUAL POR RÉGIMEN

1.1 Separando datos por régimen...
     ✓ Días en CALMA: 2998
     ✓ Días en 